In [47]:
import pygls.lsp.client as lsp_client
from pygls import uris

import lsprotocol.types as lsp_types
from typing import Dict, List, Type
import sys
import os
import pathlib

In [48]:
# https://github.com/openlawlibrary/pygls/blob/8e1e8fa3b1ab16fcb804f7d5330ece2e5583206b/tests/client.py#L162
class LanguageClient(lsp_client.BaseLanguageClient):
    """Language client used to drive test cases."""

    def __init__(
        self,
        *args,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)

        self.diagnostics: Dict[str, List[lsp_types.Diagnostic]] = {}
        """Used to hold any recieved diagnostics."""

        self.messages: List[lsp_types.ShowMessageParams] = []
        """Holds any received ``window/showMessage`` requests."""

        self.log_messages: List[lsp_types.LogMessageParams] = []
        """Holds any received ``window/logMessage`` requests."""

In [51]:
text_doc_caps = lsp_types.TextDocumentClientCapabilities(
    # synchronization=lsp_types.TextDocumentSyncClientCapabilities(),
    # completion=lsp_types.CompletionClientCapabilities(),
    hover=lsp_types.HoverClientCapabilities(),
    definition=lsp_types.DefinitionClientCapabilities(),
    declaration=lsp_types.DeclarationClientCapabilities(),
    implementation=lsp_types.ImplementationClientCapabilities(),
    # publish_diagnostics=lsp_types.PublishDiagnosticsClientCapabilities(),  # TODO: add diagnostics capability
    # type_definition=lsp_types.TypeDefinitionClientCapabilities(),
)

workspace_caps = lsp_types.WorkspaceClientCapabilities(
    apply_edit=True,
    workspace_edit=lsp_types.WorkspaceEditClientCapabilities(),
    diagnostics=lsp_types.DiagnosticClientCapabilities()
)

client_capabilities = lsp_types.ClientCapabilities(
    workspace=workspace_caps,
    text_document=text_doc_caps
)

client_trace = lsp_types.TraceValues("messages")

rust_project_dir = "/mnt/vm_data/pythonProjects/rust_analyzer_client/rust_lsp_test/"

init_params = lsp_types.InitializeParams(
    root_path=rust_project_dir,
    capabilities=client_capabilities,
    trace=client_trace,
    # process_id=1302555  # the rust-analyzer server opened by vscode????
)

In [ ]:
client = LanguageClient("rust_lsp_client", "0.1")


@client.feature(lsp_types.TEXT_DOCUMENT_PUBLISH_DIAGNOSTICS)
def publish_diagnostics(
    client: LanguageClient, params: lsp_types.PublishDiagnosticsParams
):
    client.diagnostics[params.uri] = params.diagnostics


@client.feature(lsp_types.WINDOW_LOG_MESSAGE)
def log_message(client: LanguageClient, params: lsp_types.LogMessageParams):
    client.log_messages.append(params)

    levels = ["ERROR: ", "WARNING: ", "INFO: ", "LOG: "]
    log_level = levels[params.type.value - 1]

    print(log_level, params.message)


@client.feature(lsp_types.WINDOW_SHOW_MESSAGE)
def show_message(client: LanguageClient, params):
    client.messages.append(params)

In [50]:
await client.start_io("rust-analyzer")

In [52]:
init_result = await client.initialize_async(init_params)
init_result

InitializeResult(capabilities=ServerCapabilities(position_encoding='utf-16', text_document_sync=TextDocumentSyncOptions(open_close=True, change=<TextDocumentSyncKind.Incremental: 2>, will_save=None, will_save_wait_until=None, save=SaveOptions(include_text=None)), notebook_document_sync=None, completion_provider=CompletionOptions(trigger_characters=[':', '.', "'", '('], all_commit_characters=None, resolve_provider=None, completion_item=CompletionOptionsCompletionItemType(label_details_support=False), work_done_progress=None), hover_provider=True, signature_help_provider=SignatureHelpOptions(trigger_characters=['(', ',', '<'], retrigger_characters=None, work_done_progress=None), declaration_provider=True, definition_provider=True, type_definition_provider=True, implementation_provider=True, references_provider=True, document_highlight_provider=True, document_symbol_provider=True, code_action_provider=True, code_lens_provider=CodeLensOptions(resolve_provider=True, work_done_progress=None)

In [53]:
client.initialized(init_params)

In [56]:
definition_provider = init_result.capabilities.definition_provider
declaration_provider = init_result.capabilities.declaration_provider

In [55]:
file = "src/main.rs"

file_uri = uris.from_fs_path(os.path.join(rust_project_dir, file))
symbol_position = lsp_types.Position(line=79, character=9)
file_uri

'file:///mnt/vm_data/pythonProjects/rust_analyzer_client/rust_lsp_test/src/main.rs'

In [45]:
response = await client.text_document_definition_async(
    lsp_types.DefinitionParams(
        text_document=lsp_types.TextDocumentIdentifier(file_uri),
        position=symbol_position,
    )
)
response

[file:///mnt/vm_data/pythonProjects/rust_analyzer_client/rust_lsp_test/src/main.rs:35:7-35:15]

In [57]:
response = await client.text_document_declaration_async(
    lsp_types.DeclarationParams(
        text_document=lsp_types.TextDocumentIdentifier(file_uri),
        position=symbol_position,
    )
)
response

[file:///mnt/vm_data/pythonProjects/rust_analyzer_client/rust_lsp_test/src/main.rs:35:7-35:15]